In [2]:
from sglang.utils import (
    execute_shell_command,
    wait_for_server,
    terminate_process,
    print_highlight,
)
import subprocess, json

In [ ]:

curl_command = """
curl -s http://localhost:30000/v1/chat/completions \
  -d '{"model": "Qwen/Qwen2-1.5B-Instruct", "messages": [{"role": "user", "content": "What is the capital of France?"}]}'
"""

response = json.loads(subprocess.check_output(curl_command, shell=True))
print(response)
print(12)

In [ ]:
response['choices']

In [ ]:
import requests

url = "http://localhost:30000/v1/chat/completions"

data = {
    "model": "Qwen/Qwen2-1.5B-Instruct",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

In [ ]:
import openai

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

response = client.chat.completions.create(
    model="Qwen/Qwen2-1.5B-Instruct",
    messages=[
        {"role": "user", "content": "where is the pain of bile duct obstruction located tell me?"},
    ],
    temperature=0.6,
    max_tokens=64,
)
print_highlight(response)

In [ ]:
import openai

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

# Use stream=True for streaming responses
response = client.chat.completions.create(
    model="Qwen/Qwen2-1.5B-Instruct",
    messages=[
        {"role": "user", "content": "what is 2+2+my heart?"},
    ],
    temperature=0,
    max_tokens=2000,
    stream=True,
)

# Handle the streaming output
for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)

In [ ]:
import requests

response = requests.post(
    "http://localhost:30000/generate",
    json={
        "text": "ٔIranians are",
        "sampling_params": {
            "temperature": 0.9,
            "max_new_tokens": 300,
        },
    },
)

print_highlight(response.json())

In [ ]:
import requests, json

response = requests.post(
    "http://localhost:30000/generate",
    json={
        "text": "The capital of France is",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 32,
        },
        "stream": True,
    },
    stream=True,
)

prev = 0
for chunk in response.iter_lines(decode_unicode=False):
    chunk = chunk.decode("utf-8")
    if chunk and chunk.startswith("data:"):
        if chunk == "data: [DONE]":
            break
        data = json.loads(chunk[5:].strip("\n"))
        output = data["text"]
        print(output[prev:], end="", flush=True)
        prev = len(output)

In [ ]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a historian who provides concise and broad answers.",
        },
        {"role": "user", "content": "Tell me about ancient Rome"},
        {
            "role": "assistant",
            "content": "Ancient Rome was a civilization centered in Italy.",
        },
        {"role": "user", "content": "What were their major achievements?"},
    ],
    temperature=0.6,  # Lower temperature for more focused responses
    max_tokens=128,  # Reasonable length for a concise response
    top_p=0.95,  # Slightly higher for better fluency
    presence_penalty=0.2,  # Mild penalty to avoid repetition
    frequency_penalty=0.2,  # Mild penalty for more natural language
    n=1,  # Single response is usually more stable
    seed=42,  # Keep for reproducibility
)

print_highlight(response.choices[0].message.content)

In [ ]:
url = "http://localhost:30000/health_generate"

response = requests.get(url)
response.text

In [ ]:
# successful update with same architecture and size

url = "http://localhost:30000/update_weights_from_disk"
data = {"model_path": "Qwen/Qwen2-1.5B-Instruct"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."
assert response.json().keys() == {"success", "message"}

In [ ]:
import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="Qwen/Qwen2-1.5B-Instruct")

In [ ]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")